In [42]:
import matplotlib.pyplot as plt
import matplotlib.path as path
import cv2
import numpy as np
import os
from os.path import join
import glob
import json
import pandas as pd
from skimage.measure import label, regionprops
from scipy.ndimage import binary_fill_holes, binary_dilation, binary_erosion
from scipy.spatial import ConvexHull
from skimage.transform import resize
from skimage.morphology import convex_hull_object, remove_small_objects

In [164]:
from sklearn.metrics import jaccard_score

def calculate_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    eps = 1e-8
    iou = np.sum(intersection) / (np.sum(union) + eps)
    return iou

def calculate_metrics(true_mask, pred_mask, iou_threshold=0.1):
    threshold = 0.5  # Set your desired threshold value

    true_labels = label(true_mask)
    pred_labels = label(pred_mask)

    true_positives = 0
    false_negatives = 0
    false_positives = 0
    visited = set()

    for j in range(1, np.max(true_labels) + 1):
        true_region = true_labels == j
        match_found = False

        for k in range(1, np.max(pred_labels) + 1):
            if k in visited:
                continue

            pred_region = pred_labels == k

            true_region_flat = true_region.flatten()
            pred_region_flat = pred_region.flatten()

            iou = jaccard_score(true_region_flat, pred_region_flat)

            if iou > iou_threshold:
                true_positives += 1
                match_found = True
                visited.add(k)
                break

        if not match_found:
            false_negatives += 1

    false_positives = np.max(pred_labels) - true_positives

    eps = 1e-8
    dice_coefficient = true_positives / (true_positives + false_negatives + eps)

    return dice_coefficient

# evaluation

In [110]:
def modifytune(mask, area=200, convex=True):
    n, m = mask.shape
    
    mask = binary_dilation(mask, iterations=2)
    mask = binary_fill_holes(mask)
    mask = binary_erosion(mask, iterations=1)

    mask = remove_small_objects(mask, area)
    
    if convex:
        label_img = label(mask)
        regions = regionprops(label_img)
        for idx in range(len(regions)):
            hull = ConvexHull(regions[idx].coords)  
            poly_points = np.array([regions[idx].coords[hull.vertices,0], regions[idx].coords[hull.vertices,1]])
            poly_points = np.transpose(poly_points)
            polygon = path.Path(poly_points)
            
            for i in range(min(poly_points[:, 0]), max(poly_points[:, 0])):
                for j in range(min(poly_points[:, 1]), max(poly_points[:, 1])):
                    if polygon.contains_point((i, j)):
                        mask[i, j] = 1
    
        
    mask = binary_erosion(mask, iterations=1)
            
    return mask

In [ ]:
srcdir = './load/TB512test_chen176'
maskdir = './load/TB512test_chen176MASK'
datadir = './contours_chen512_35ep'

namelist = sorted(glob.glob(os.path.join(srcdir, '*.png')))
ioulist = []
dicelist = []
for name in namelist:
    iid = os.path.basename(name).split('.')[0]
    mask = cv2.imread(join(maskdir, iid+'_mask.png'))
    if len(mask.shape) == 3:
        mask = mask[:,:,0]
    # nuc = getnuc(datadir, iid)
    pred = np.load(join(datadir, 'probmap', f'{iid}.npy'))
    pred = np.squeeze(pred) > 0.9
    pred = resize(pred, (512, 512))

    pred = modifytune(pred, 500, False) * 255

    pred = pred.astype('uint8')
    mask = mask.astype('uint8')

    cv2.imwrite(join(datadir, 'pred_refine', f'{iid}.png'), pred)
    
    iou = calculate_iou(pred, mask)
    ioulist.append(iou)

    dice = calculate_metrics(mask, pred, 0.5)
    dicelist.append(dice)
    

print('iou: {}, dice: {}'.format(np.mean(ioulist), np.mean(dicelist)))

# contours

In [196]:
def draw_contours(image, mask, color):
    mask = np.squeeze(mask)  # Remove singleton dimensions if present

    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return image  # Return the original image if no contours are found

    # Draw the contours on the image
    cv2.drawContours(image, contours, -1, color, 4)

    return image

In [199]:


namelist = sorted(glob.glob(os.path.join(srcdir, '*.png')))

for name in namelist:
    iid = os.path.basename(name).split('.')[0]
    mask = cv2.imread(join(maskdir, iid+'_mask.png'))
    if len(mask.shape) == 3:
        mask = mask[:,:,0]
    # nuc = getnuc(datadir, iid)
    pred = cv2.imread(join(datadir, 'pred_refine', f'{iid}.png'), 0)

    img = cv2.imread(join(srcdir, f'{iid}.png'))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    overlay = draw_contours(img, mask, (255, 255, 0))
    overlay = draw_contours(overlay, pred, (0, 255, 0))

    plt.imsave(f'{datadir}/image_{iid}.png', overlay)